In [47]:
# Importing the libraries from the base notebook
%run Binocular.ipynb

In [101]:
def growthMCSim(input_df, years, sims = 1000):
    """
    growthMCSim takes in the recent prices dataframe. Historic prices should be sorted so that the most recent prices are at the top of the
    dataframe
    """
    # Calculating MoM growth and parameters of the Normal Distribution
    currentPrice = input_df["Price"].iloc[0]
    input_df.sort_values("MonthEnding", inplace = True)
    input_df["Growth"] = input_df["Price"].pct_change()
    std = input_df["Growth"].std()
    mean = input_df["Growth"].mean()
    
    # Creating list for the MCSim based on input parameters
    simvals = []
    
    for simNum in range(sims):
        simvals.append([currentPrice])
        for mth in range(years*12):
            simvals[simNum].append(simvals[simNum][-1]*(1+np.random.normal(mean, std)))
    
    simvals_pd = pd.DataFrame(simvals)
    
    return simvals_pd

In [25]:
def getMortgageRates(zipCode, propertyValue, loanAmount = -1, lengthofLoan = 30, creditScore = 700):
    '''
    getMortgageRates calls the Zillow getCurrentRates API and returns the APR using the given parameters. If there was an error that wasn't
    caught by an exception in retrieving the mortage rate, -1 will be returned.
        The following arguments are required:
            zipCode (as a string)
            propertyValue
        The following arguments are optional and their corresponding default values will be used:
            creditScore = 700
            loanAmount = 80% of propertyValue

            lengthofLoan = 30 (Must be 30, 20, 15, or 10 years)
    '''
    # Initializing Mortgage Rate
    mortgageRate = -1
    
    # Retrieving Zillow API Key
    partnerID = os.getenv("ZILLOW_API_KEY")
    
    # Checking zipCode
    # Enter code here
    
    # Checking propertyValue
    if propertyValue <=0:
        raise Exception ("Your property value must be a positive number")
    
    # Checking loanAmount
    if loanAmount == -1:
        loanAmount = propertyValue * 0.8
    elif loanAmount <= 0:
        raise Exception("Your loan amount must be a positive number")
    elif loanAmount > propertyValue:
        raise Exception("Your loan amount cannot exceed your property value")
    
    # Converting length of loan to loan program
    if lengthofLoan == 30:
        loanProgram = 'Fixed30Year'
    elif lengthofLoan == 20:
        loanProgram = 'Fixed20Year'
    elif lengthofLoan == 15:
        loanProgram = 'Fixed15Year'
    elif lengthofLoan == 10:
        loanProgram = 'Fixed10Year'
    else:
        raise Exception("The number you entered for the lengthofLoan was invalid. Please enter 30, 20, 15, or 10 for the length of the loan (in years)")
    
    # Converting credit score to credit score bucket (as defined by Zillow)
    if creditScore < 300:
        raise Exception("The Credit Score you entered was invalid")
    elif creditScore < 680:
        creditScoreBucket = "Low"
    elif creditScore < 740:
        creditScoreBucket = "High"
    elif creditScore < 850:
        creditScoreBucket = "VeryHigh"
    else:
        raise Exception("The Credit Score you entered was invalid")
    
    
    base_url = 'https://mortgageapi.zillow.com/getCurrentRates'
    payload = {
        'partnerId':partnerID,
        'queries.1.propertyBucket.location.zipCode':zipCode,
        'queries.1.propertyBucket.propertyValue':str(propertyValue),
        'queries.1.propertyBucket.loanAmount':str(loanAmount),
        'queries.1.RateQuery.creditScoreBucket':creditScoreBucket,
        'queries.1.RateQuery.program':loanProgram
    }
    mortgage_data = requests.get(base_url, params = payload).json()
    
    return mortgageRate

In [115]:
def rental_roi_calc(purchase_price, mortgage_apr, rent, expenses, downpayment = -1, zipcode = "", years = 10):
    """
    Function Description
    """
    zc_path = Path("Data/ZillowRegions_SF.csv")
    zc_df = pd.read_csv(zc_path)
    
    # Checking to see if the zipcode is in the list of Zillow regions. If not, the SF average will be used
    zipcode = str(zipcode)
    if (zc_df["region"].isin([zipcode]).any()):
        zillReg = zc_df.loc[zc_df["region"] == zipcode,"region_id"].iloc[0]
    else:
        print("You either did not provide a zipcode or the zipcode you entered could not be found. The San Francisco average historic prices will be used")
        zillReg = 20330
    
    # Retrieving history of single family home prices using Quandl API
    quandl_key = os.getenv("QUANDL_API_KEY")
    request_url = f"https://www.quandl.com/api/v3/datatables/ZILLOW/DATA?region_id={str(zillReg)}&indicator_id=ZSFH&api_key={quandl_key}"
    historic_prices_json = requests.get(request_url).json()
    
    # Parsing data and reformatting dataFrame to prep for Monte Carlo simulation
    historic_prices = pd.DataFrame(historic_prices_json["datatable"]["data"])
    columns = ["ZIndicator", "ZRegion", "MonthEnding", "Price"]
    historic_prices.columns = columns
    # Grabbing the last 10 years of data
    recent_prices = historic_prices.head(120)
    
    # Calculate the median value of the expected price of the property in the number of years specified
    sim_pd = growthMCSim(recent_prices, years)
    median = sim_pd[years*12].median()
    
    return median

In [119]:
sim_pd = rental_roi_calc(0,0,0,0, zipcode = 94109)

/Users/albertkong/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/albertkong/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [120]:
sim_pd

5291673.878071604

,ZIndicator,ZRegion,MonthEnding,Price
0,ZSFH,20330,2020-09-30,1491108.0
1,ZSFH,20330,2020-08-31,1483504.0
2,ZSFH,20330,2020-07-31,1480127.0
3,ZSFH,20330,2020-06-30,1496695.0
4,ZSFH,20330,2020-05-31,1503644.0
5,ZSFH,20330,2020-04-30,1533941.0
6,ZSFH,20330,2020-03-31,1535882.0
7,ZSFH,20330,2020-02-29,1539101.0
8,ZSFH,20330,2020-01-31,1535710.0
9,ZSFH,20330,2019-12-31,1531203.0


In [41]:
housing_data["Growth"].std()

In [96]:
mc = growthMCSim(housing_data, 10)

In [97]:
pd.DataFrame(mc)

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,1491108.0,1.484633e+06,1.472496e+06,1.496442e+06,1.492760e+06,1.511043e+06,1.528900e+06,1.518913e+06,1.527279e+06,1.517798e+06,...,2.514605e+06,2.525007e+06,2.559120e+06,2.580100e+06,2.584507e+06,2.611954e+06,2.649122e+06,2.642323e+06,2.683033e+06,2.698512e+06
1,1491108.0,1.508385e+06,1.517927e+06,1.544935e+06,1.537237e+06,1.536819e+06,1.534008e+06,1.539730e+06,1.533783e+06,1.542061e+06,...,3.295097e+06,3.275755e+06,3.280959e+06,3.275884e+06,3.323575e+06,3.334555e+06,3.395079e+06,3.418983e+06,3.485455e+06,3.497045e+06
2,1491108.0,1.488447e+06,1.518118e+06,1.522779e+06,1.550261e+06,1.576749e+06,1.592393e+06,1.601591e+06,1.639285e+06,1.637896e+06,...,2.497106e+06,2.516663e+06,2.545467e+06,2.557752e+06,2.574156e+06,2.561095e+06,2.587034e+06,2.580491e+06,2.592569e+06,2.627031e+06
3,1491108.0,1.486619e+06,1.489376e+06,1.507242e+06,1.495551e+06,1.509049e+06,1.499276e+06,1.531503e+06,1.561779e+06,1.565132e+06,...,2.645742e+06,2.663942e+06,2.669716e+06,2.655982e+06,2.634436e+06,2.661448e+06,2.699688e+06,2.655516e+06,2.716222e+06,2.730148e+06
4,1491108.0,1.484600e+06,1.462800e+06,1.467212e+06,1.479662e+06,1.482545e+06,1.500228e+06,1.521230e+06,1.552725e+06,1.556274e+06,...,2.526559e+06,2.542716e+06,2.545682e+06,2.549528e+06,2.601707e+06,2.622261e+06,2.636803e+06,2.590660e+06,2.590309e+06,2.594242e+06
5,1491108.0,1.500336e+06,1.533545e+06,1.569219e+06,1.574369e+06,1.598185e+06,1.603220e+06,1.598309e+06,1.632370e+06,1.627640e+06,...,2.689472e+06,2.684207e+06,2.658461e+06,2.653382e+06,2.651452e+06,2.703453e+06,2.709216e+06,2.696725e+06,2.725583e+06,2.753790e+06
6,1491108.0,1.501992e+06,1.510611e+06,1.513858e+06,1.503230e+06,1.508194e+06,1.533280e+06,1.532136e+06,1.546689e+06,1.538229e+06,...,2.218925e+06,2.205251e+06,2.241642e+06,2.209441e+06,2.219199e+06,2.235189e+06,2.271269e+06,2.301552e+06,2.313722e+06,2.301365e+06
7,1491108.0,1.491913e+06,1.494745e+06,1.483714e+06,1.486840e+06,1.497277e+06,1.492069e+06,1.504402e+06,1.506370e+06,1.539876e+06,...,3.191504e+06,3.187929e+06,3.217629e+06,3.200410e+06,3.203519e+06,3.239902e+06,3.276222e+06,3.256073e+06,3.289481e+06,3.333937e+06
8,1491108.0,1.523053e+06,1.540333e+06,1.531298e+06,1.537970e+06,1.558951e+06,1.573524e+06,1.577487e+06,1.568154e+06,1.587019e+06,...,2.518605e+06,2.550785e+06,2.555680e+06,2.594122e+06,2.632457e+06,2.631609e+06,2.634436e+06,2.613082e+06,2.633779e+06,2.678635e+06
9,1491108.0,1.490130e+06,1.512343e+06,1.529171e+06,1.518726e+06,1.516938e+06,1.513985e+06,1.506039e+06,1.510484e+06,1.510962e+06,...,2.936767e+06,2.913292e+06,2.948386e+06,2.995388e+06,3.000218e+06,3.019362e+06,3.014914e+06,3.047798e+06,3.063798e+06,3.115791e+06


In [60]:
years = 10
np.random.normal(0,1,years*12)

array([-1.30670270e+00,  4.14729295e-01, -7.04854018e-01,  6.86518838e-01,
        1.06865595e+00, -2.33337217e-01,  2.24979907e-01, -2.58779670e-01,
        7.36811735e-01,  7.76371284e-01,  2.14663703e-01, -5.52281097e-01,
       -3.62133381e-01, -5.79411306e-01, -1.43439054e+00,  8.27522406e-01,
       -9.85286425e-01, -8.03341570e-01,  6.11418174e-01,  1.62987344e-02,
        7.26449256e-01, -1.10661930e+00,  7.96003766e-01, -1.47227588e+00,
       -5.97837429e-01, -5.18398480e-01, -8.01309246e-01, -1.06927380e+00,
        2.33331817e-01,  4.12673098e-01,  1.47811284e-01,  8.41680990e-02,
        1.41348284e+00, -2.37325788e+00, -5.75410418e-01,  1.84423296e+00,
        1.16233673e+00, -1.43025686e+00, -2.70629290e-01,  9.04900569e-01,
       -3.07852562e-01, -1.44940967e+00,  2.95249288e-01,  2.15341605e+00,
       -1.12503427e+00, -8.97992193e-01, -7.50984436e-01, -2.06715069e-03,
       -1.21427170e+00, -2.11774964e+00, -2.99114162e-01, -3.53417964e-01,
       -9.21760772e-01,  

In [ ]:
def purchase_roi_calc(purchase_price, mortgage_apr, downpayment = -1, zipcode = "", years = 10):
    """
    Function Description
    """
    
    roi = 0
    
    return roi

In [ ]:
def rent_rec(purchase_price, downpayment = -1, zipcode = "", years = 10):
    """
    Function Description
    """
    
    rent = 0
    
    return rent